In [ ]:
import os
import time
import glob
import gdown
import torch
import torch.nn.functional as F
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import skimage.feature as feature
import xlwings as xw
import torchvision.transforms as transforms

#libraries for yolo
from pytorchyolo.models import load_model
from pytorchyolo.utils.transforms import Resize, DEFAULT_TRANSFORMS
from pytorchyolo.utils.utils import non_max_suppression

from matplotlib.ticker import (FormatStrFormatter, AutoMinorLocator, FuncFormatter, )

In [ ]:
def download_weights():
    model_file=[
        'yolo_face_sthanhng.weights',
        'yolo_face_sthanhng.cfg'
    ]
    
    gdrive_url=[
        'https://drive.google.com/uc?id=1utquM5TAnfIa1Aq0X9fCvrllHiTWazdD',
        'https://drive.google.com/uc?id=1CPUZlYL5ik4d9y6oCyzi0930KgzawI6V'
    ]
    
    cwd=os.getcwd() 
    if 'weights' in os.listdir(cwd):
        for i in range(len(model_file)):
            if model_file[i] in os.listdir(os.path.join(cwd, 'weights')):
                print(model_file[i] + ':: status : file already exists')
            else:
                gdown.download(gdrive_url[i],os.path.join(cwd, 'weights', model_file[i]), quiet=False)
    else:
        os.makedirs(os.path.join(cwd,'weights'))
        for i in range(len(model_file)):
            gdown.download(gdrive_url[i], os.path.join(cwd, 'weights', model_file[i]), quiet=False)  

In [ ]:
# download the necessary weights for YOLO-Face
download_weights()

## YOLOFace with FGSM

In [ ]:
# Patterned after FGSM tutorial (https://pytorch.org/tutorials/beginner/fgsm_tutorial.html)
# Define what device we are using
print("CUDA Available: ", torch.cuda.is_available())
device, model = load_model('./weights/yolo_face_sthanhng.cfg', "./weights/yolo_face_sthanhng.weights")

# Set the model in evaluation mode. In this case this is for the Dropout layers
model.eval()

epsilons = [0, .05]
use_cuda=True

In [ ]:
# FGSM attack code
def fgsm_attack(image, epsilon, data_grad, x1, y1, x2, y2):
    # Collect the element-wise sign of the data gradient
    image = image
    sign_data_grad = data_grad.sign()
    # Create the perturbed image by adjusting each pixel of the input image
    perturbed_image = image
    perturbed_image[:, :, y1:y2, x1:x2] = perturbed_image[:, :, y1:y2, x1:x2] + epsilon * sign_data_grad[:, :, y1:y2, x1:x2] # apply it only to the face region
    # Adding clipping to maintain [0,1] range
    perturbed_image = torch.clamp(perturbed_image, 0, 1)
    # Return the perturbed image
    return perturbed_image

In [ ]:
class LocalBinaryPatterns:
  def __init__(self, numPoints, radius):
    self.numPoints = numPoints
    self.radius = radius

  def describe(self, image, eps = 1e-7):
    lbp = feature.local_binary_pattern(image, self.numPoints, self.radius, method="uniform")
    (hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, self.numPoints+3), range=(0, self.numPoints + 2))

    # Normalize the histogram
    hist = hist.astype('float')
    hist /= (hist.sum() + eps)

    return hist, lbp

# From https://medium.com/mlearning-ai/how-to-plot-color-channels-histogram-of-an-image-in-python-using-opencv-40022032e127
# Extracts image's color channel
def extract_color_channel(path, face_index, image):
    # BGR Image Color Conversion
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hls = cv2.cvtColor(image, cv2.COLOR_BGR2HLS)
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    ycrcb = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)

    # RGB Image Histogram
    red_hist = cv2.calcHist([rgb], [0], None, [256], [0, 255])
    green_hist = cv2.calcHist([rgb], [1], None, [256], [0, 255])
    blue_hist = cv2.calcHist([rgb], [2], None, [256], [0, 255])

    # HSV Image Histogram
    hue_hist_HSV = cv2.calcHist([hsv], [0], None, [256], [0, 255])
    saturation_hist_HSV = cv2.calcHist([hsv], [1], None, [256], [0, 255])
    value_hist = cv2.calcHist([hsv], [2], None, [256], [0, 255])

    # HLS Image Histogram
    hue_hist_HLS = cv2.calcHist([hls], [0], None, [256], [0, 255])
    lightness_hist_HLS = cv2.calcHist([hls], [1], None, [256], [0, 255])
    saturation_hist_HLS = cv2.calcHist([hls], [2], None, [256], [0, 255])

    # LAB Image Histogram
    lightness_hist_LAB = cv2.calcHist([lab], [0], None, [256], [0, 255])
    a_hist_LAB = cv2.calcHist([lab], [1], None, [256], [0, 255])
    b_hist_LAB = cv2.calcHist([lab], [2], None, [256], [0, 255])

    # YCrCb Image Histogram
    y_hist = cv2.calcHist([ycrcb], [0], None, [256], [0, 255])
    cr_hist = cv2.calcHist([ycrcb], [1], None, [256], [0, 255])
    cb_hist = cv2.calcHist([ycrcb], [2], None, [256], [0, 255])

    # RGB Image Plot
    plt.subplot(4, 1, 1)
    plt.imshow(rgb)
    plt.title('RGB Image')
    plt.xticks([])
    plt.yticks([])

    plt.subplot(4, 1, 2)
    plt.plot(red_hist, color='r')
    plt.xlim([0, 255])
    plt.ylim([0, 500])
    plt.title('Red Histogram')

    plt.subplot(4, 1, 3)
    plt.plot(green_hist, color='g')
    plt.xlim([0, 255])
    plt.ylim([0, 500])
    plt.title('Green Histogram')

    plt.subplot(4, 1, 4)
    plt.plot(blue_hist, color='b')
    plt.xlim([0, 255])
    plt.ylim([0, 500])
    plt.title('Blue Histogram')

    plt.tight_layout()
    plt.show()

    r, g, b = cv2.split(rgb)
    
    r_hist = cv2.calcHist([r], [0], None, [26], [0, 255])
    r_hist = r_hist.ravel()
    r_hist = r_hist.astype('float')
    r_hist /= r_hist.sum()
    
    g_hist = cv2.calcHist([g], [0], None, [26], [0, 255])
    g_hist = g_hist.ravel()
    g_hist = g_hist.astype('float')
    g_hist /= g_hist.sum()
    
    b_hist = cv2.calcHist([b], [0], None, [26], [0, 255])
    b_hist = b_hist.ravel()
    b_hist = b_hist.astype('float')
    b_hist /= b_hist.sum()

    # HSV Image Plot
    plt.subplot(4, 1, 1)
    plt.imshow(hsv)
    plt.title('HSV Image')
    plt.xticks([])
    plt.yticks([])

    plt.subplot(4, 1, 2)
    plt.plot(hue_hist_HSV, color='c')
    plt.xlim([0, 255])
    plt.ylim([0, 2500])
    plt.title('Hue Histogram')

    plt.subplot(4, 1, 3)
    plt.plot(saturation_hist_HSV, color='m')
    plt.xlim([0, 255])
    plt.ylim([0, 1000])
    plt.title('Saturation Histogram')

    plt.subplot(4, 1, 4)
    plt.plot(value_hist, color='y')
    plt.xlim([0, 255])
    plt.ylim([0, 1000])
    plt.title('Value Histogram')

    plt.tight_layout()
    plt.show()
    
    h, s, v = cv2.split(hsv)
    
    h_hist_HSV = cv2.calcHist([h], [0], None, [18], [0, 180])
    h_hist_HSV = h_hist_HSV.ravel()
    h_hist_HSV = h_hist_HSV.astype('float')
    h_hist_HSV /= h_hist_HSV.sum()
    
    s_hist_HSV = cv2.calcHist([s], [0], None, [26], [0, 256])
    s_hist_HSV = s_hist_HSV.ravel()
    s_hist_HSV = s_hist_HSV.astype('float')
    s_hist_HSV /= s_hist_HSV.sum()
    
    v_hist_HSV = cv2.calcHist([v], [0], None, [26], [0, 256])
    v_hist_HSV = v_hist_HSV.ravel()
    v_hist_HSV = v_hist_HSV.astype('float')
    v_hist_HSV /= v_hist_HSV.sum()
    
    # HLS Image Plot
    plt.subplot(4, 1, 1)
    plt.imshow(hls)
    plt.title('HLS Image')
    plt.xticks([])
    plt.yticks([])

    plt.subplot(4, 1, 2)
    plt.plot(hue_hist_HLS, color='r')
    plt.xlim([0, 255])
    plt.ylim([0, 2500])
    plt.title('Hue Histogram')

    plt.subplot(4, 1, 3)
    plt.plot(lightness_hist_HLS, color='g')
    plt.xlim([0, 255])
    plt.ylim([0, 1000])
    plt.title('Lightness Histogram')

    plt.subplot(4, 1, 4)
    plt.plot(saturation_hist_HLS, color='b')
    plt.xlim([0, 255])
    plt.ylim([0, 1000])
    plt.title('Saturation Histogram')

    plt.tight_layout()
    plt.show()

    h, l, s = cv2.split(hls)
    
    h_hist_HSL = cv2.calcHist([h], [0], None, [18], [0, 180])
    h_hist_HSL = h_hist_HSL.ravel()
    h_hist_HSL = h_hist_HSL.astype('float')
    h_hist_HSL /= h_hist_HSL.sum()
    
    l_hist_HSL = cv2.calcHist([l], [0], None, [26], [0, 256])
    l_hist_HSL = l_hist_HSL.ravel()
    l_hist_HSL = l_hist_HSL.astype('float')
    l_hist_HSL /= l_hist_HSL.sum()
    
    s_hist_HSL = cv2.calcHist([s], [0], None, [26], [0, 256])
    s_hist_HSL = s_hist_HSL.ravel()
    s_hist_HSL = s_hist_HSL.astype('float')
    s_hist_HSL /= s_hist_HSL.sum()
    
    # LAB Image Plot
    plt.subplot(4, 1, 1)
    plt.imshow(lab)
    plt.title('LAB Image')
    plt.xticks([])
    plt.yticks([])

    plt.subplot(4, 1, 2)
    plt.plot(lightness_hist_LAB, color='c')
    plt.xlim([0, 255])
    plt.ylim([0, 1000])
    plt.title('Lightness Histogram')

    plt.subplot(4, 1, 3)
    plt.plot(a_hist_LAB, color='m')
    plt.xlim([0, 255])
    plt.ylim([0, 20000])
    plt.title('A Histogram')

    plt.subplot(4, 1, 4)
    plt.plot(b_hist_LAB, color='y')
    plt.xlim([0, 255])
    plt.ylim([0, 20000])
    plt.title('B Histogram')

    plt.tight_layout()
    plt.show()
    
    l, a, b = cv2.split(lab)
    
    l_hist_LAB = cv2.calcHist([l], [0], None, [26], [0, 256])
    l_hist_LAB = l_hist_LAB.ravel()
    l_hist_LAB = l_hist_LAB.astype('float')
    l_hist_LAB /= l_hist_LAB.sum()
    
    a_hist_LAB = cv2.calcHist([a], [0], None, [26], [0, 256])
    a_hist_LAB = a_hist_LAB.ravel()
    a_hist_LAB = a_hist_LAB.astype('float')
    a_hist_LAB /= a_hist_LAB.sum()
    
    b_hist_LAB = cv2.calcHist([b], [0], None, [26], [0, 256])
    b_hist_LAB = b_hist_LAB.ravel()
    b_hist_LAB = b_hist_LAB.astype('float')
    b_hist_LAB /= b_hist_LAB.sum()
    
    # YCrCb Image Plot
    plt.subplot(4, 1, 1)
    plt.imshow(ycrcb)
    plt.title('YCrCb Image')
    plt.xticks([])
    plt.yticks([])

    plt.subplot(4, 1, 2)
    plt.plot(y_hist, color='r')
    plt.xlim([0, 255])
    plt.ylim([0, 1000])
    plt.title('Y Histogram')

    plt.subplot(4, 1, 3)
    plt.plot(cr_hist, color='g')
    plt.xlim([0, 255])
    plt.ylim([0, 20000])
    plt.title('Cr Histogram')

    plt.subplot(4, 1, 4)
    plt.plot(cb_hist, color='b')
    plt.xlim([0, 255])
    plt.ylim([0, 20000])
    plt.title('Cb Histogram')

    plt.tight_layout()
    plt.show()
    
    y, cr, cb = cv2.split(ycrcb)
    
    y_hist = cv2.calcHist([y], [0], None, [26], [0, 256])
    y_hist = y_hist.ravel()
    y_hist = y_hist.astype('float')
    y_hist /= y_hist.sum()
    
    cr_hist = cv2.calcHist([cr], [0], None, [26], [0, 256])
    cr_hist = cr_hist.ravel()
    cr_hist = cr_hist.astype('float')
    cr_hist /= cr_hist.sum()
    
    cb_hist = cv2.calcHist([cb], [0], None, [26], [0, 256])
    cb_hist = cb_hist.ravel()
    cb_hist = cb_hist.astype('float')
    cb_hist /= cb_hist.sum()
    
    face_index = str(face_index)
    rows = itertools.zip_longest([path], [face_index], r_hist, g_hist, b_hist, h_hist_HSV, s_hist_HSV, v_hist_HSV, h_hist_HSL, s_hist_HSL, l_hist_HSL, l_hist_LAB, a_hist_LAB, b_hist_LAB, y_hist, cr_hist, cb_hist)
    
    with open("color.csv", "a", newline = "") as f:
        if os.stat("color.csv").st_size == 0:
            csv.writer(f).writerow(["Path", "Face Index", "Red", "Green", "Blue", "Hue_HSV", "Saturation_HSV", "Value_HSV", "Hue_HSL", "Saturation_HSL", "Lightness_HSL", "Lightness_LAB", "A_LAB", "B_LAB", "Y", "Cr", "Cb"])
        csv.writer(f).writerows(rows)

# From https://medium.com/mlearning-ai/color-shape-and-texture-feature-extraction-using-opencv-cb1feb2dbd73
# Extracts Local Binary Pattern (Texture) of an image
def extract_lbp(path, face_index, image):
    # reads the input image as a grayscale image
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    desc = LocalBinaryPatterns(24, 8)
    lbp_hist, lbp_img = desc.describe(gray)

    plt.imshow(lbp_img, cmap = plt.get_cmap('gray'))
    plt.show()
    
    #lbp_hist = cv2.calcHist([lbp_img], [0], None, [256], [0, 256])
    lbp_hist = lbp_hist.ravel()
    lbp_hist = lbp_hist.astype('float')
    lbp_hist /= lbp_hist.sum()
    
    face_index = str(face_index)
    rows = itertools.zip_longest([path], [face_index], lbp_hist)
    
    with open("lbp.csv", "a", newline = "") as f:
        if os.stat("lbp.csv").st_size == 0:
            csv.writer(f).writerow(["Path", "Face Index", "LBP"])
        csv.writer(f).writerows(rows)
    
# From https://docs.opencv.org/4.x/d2/d2c/tutorial_sobel_derivatives.html and https://gist.github.com/rahit/c078cabc0a48f2570028bff397a9e154
def extract_gradients(path, face_index, image):
    # Uses the Sobel Filter to extract the gradients of an image
    # reads the input image, then converts BGR color space to RGB
    # img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # compute the 1st order Sobel derivative in X-direction
    sobelx = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=5)

    # compute the 1st order Sobel derivative in Y-direction
    sobely = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=5)
    
    # combine sobelx and sobely to form sobel
    sobel = sobelx + sobely

    # display sobelx, sobely, and sobel
    plt.imshow(sobelx, cmap = "gray")
    plt.show()
    plt.imshow(sobely, cmap = "gray")
    plt.show()
    plt.imshow(sobel, cmap = "gray")
    plt.show()
    
    sobelx_hist = cv2.calcHist([np.float32(sobelx)], [0], None, [26], [0, 256])
    sobelx_hist = sobelx_hist.ravel()
    sobelx_hist = sobelx_hist.astype('float')
    sobelx_hist /= sobelx_hist.sum()
    
    sobely_hist = cv2.calcHist([np.float32(sobely)], [0], None, [26], [0, 256])
    sobely_hist = sobely_hist.ravel()
    sobely_hist = sobely_hist.astype('float')
    sobely_hist /= sobely_hist.sum()
    
    sobel_hist = cv2.calcHist([np.float32(sobel)], [0], None, [26], [0, 256])
    sobel_hist = sobel_hist.ravel()
    sobel_hist = sobel_hist.astype('float')
    sobel_hist /= sobel_hist.sum()
    
    face_index = str(face_index)
    rows = itertools.zip_longest([path], [face_index], sobelx_hist, sobely_hist, sobel_hist)
    
    with open("gradient.csv", "a", newline = "") as f:
        if os.stat("gradient.csv").st_size == 0:
            csv.writer(f).writerow(["Path", "Face Index", "Sobel X", "Sobel Y", "Sobel"])
        csv.writer(f).writerows(rows)

def extract_image_attributes(path, face_index, image):
    extract_color_channel(path, face_index, image)
    extract_lbp(path, face_index, image)
    extract_gradients(path, face_index, image)

In [ ]:
def detach_cpu(image):
    return image.detach().cpu()

# convert 1x3x416x416 to 416x416x3
def reshape_image(image):
    return np.transpose(np.squeeze(image), (1 ,2, 0))

# convert 1x3x416x416 tensor to 416x416x3 numpy image
def tensor_to_image(image):
    return np.transpose(image.detach().cpu().squeeze().numpy(), (1, 2, 0))

In [ ]:
import minepsilon as minE

In [ ]:
FOLDER_PATH = os.path.join(os.getcwd(), 'images')
print(FOLDER_PATH)

def pipeline( model, device):
    
    torch.autograd.set_detect_anomaly(True)
    
    df = pd.DataFrame() # dataframe storing the dataset
    row = {} #the information/columns for a single row in the dataset is stored here
    
    # Loop over all examples in test set
    for path in glob.glob(os.path.join(FOLDER_PATH, 'couple.jpg')):
        row['path'] = path
        print(path)
        
        # read and transform the image from the path
        data = cv2.imread(path)  # read the image
        data = cv2.cvtColor(data, cv2.COLOR_BGR2RGB) #change to rgb
        data = transforms.Compose([DEFAULT_TRANSFORMS,Resize(416)])((data, np.zeros((1, 5))))[0].unsqueeze(0) # transform the image
    
        data = data.to(device)
        
        print('Input')
        print(data.shape)
        plt.imshow(tensor_to_image(data))
        plt.show()
        
        # Set requires_grad attribute of tensor. Important for Attack
        data.requires_grad = True
        
        # Forward pass the data through the model
        output = model(data)
#         print('Model Output')
#         print(output)
#         print(output.shape)
        
        # call non max suppression
        nms, nms_output = non_max_suppression(output, 0.5, 0.5) #conf_thres and iou_thres = 0.5
        print('NMS')
        print(nms)
        print(nms_output)
        
        # loop through each of the faces in the image
        for face_index, face_row in enumerate(nms_output[0]): #nms_output[0] because the model is designed to take in several images at a time from the dataloader but we are only loading the image one at a time
            row['face_index'] = face_index
            print('Face ', face_index)
            print(face_row)

            # Calculate the loss
            #TODO: check if this is correct when determining what should be the ground truth
            loss = F.binary_cross_entropy(face_row[5:], torch.tensor([0.])) # index 0 - face, index 1 - back. only the classification loss was used
            
            # get the coordinate of the face bounding box
            #(x1, y1) lower left, (x2, y2) upper right
            x1 = int(np.floor((face_row[0] - face_row[2] / 2).detach().cpu().numpy()))
            y1 = int(np.floor((face_row[1] - face_row[3] / 2).detach().cpu().numpy()))
            x2 = int(np.ceil((face_row[0] + face_row[2] / 2).detach().cpu().numpy()))
            y2 = int(np.ceil((face_row[1] + face_row[3] / 2).detach().cpu().numpy()))
            
            row['x1'], row['y1'], row['x2'], row['y2'] = x1, y1, x2, y2
            
            print('Cropped')
            print(x1, y1, x2, y2)
            cropped_image = detach_cpu(data)[:, :, y1:y2, x1:x2] #get the first dimension, the channels, and crop it
            cropped_image = tensor_to_image(cropped_image) #reshape the image to (w/h, h/w, channel)
            plt.imshow(cropped_image)
            plt.show()
            
            #TODO: Jay - extract image attributes here
            # extract the image attributes from  the 'cropped_image' variable
            # save the attributes as row['<column name in the dataset>'] = <data> (see examples above for reference)
            
            #extract_image_attributes(cropped_image)
            extract_image_attributes(path, face_index, cropped_image)
            
            print('Resized')
            cropped_resized_image = np.transpose(transforms.Compose([DEFAULT_TRANSFORMS,Resize(128)])((cropped_image, np.zeros((1, 5))))[0], (1, 2, 0))
            plt.imshow(cropped_resized_image)
            plt.show()
            
            #TODO: Aaron - perform face segmentation here
            # using the 'cropped_resized_image'
            
            # Zero all existing gradients
            model.zero_grad()
            data.grad = None

            # Calculate gradients of model in backward pass
            loss.backward(retain_graph=True) #TODO: Amos - check if this is correct
            
            # Collect datagrad
            data_grad = data.grad.data
            print('Gradient')
            print(data_grad.shape)
            plt.imshow(np.transpose(np.clip(data_grad.squeeze(0).numpy(), 0, 1), (1, 2, 0)))
            plt.show()
            
            # TODO - Amos - determine the value of epsilon by calling fgsm_attack and changing the value of epsilon (see code below)
            # the value of data(image) and data_grad remains constant diba
            print("Calculating min epsilon for YuNet...")
            yn_min_e = minE.min_model_eps(data.clone().detach(), data_grad.clone().detach(), minE.yn_det_fn, (x1, y1, x2, y2))
            print("Calculating min epsilon for MediaPipe...")
            mp_min_e = minE.min_model_eps(data.clone().detach(), data_grad.clone().detach(), minE.mp_det_fn, (x1, y1, x2, y2))
            print("Calculating min epsilon for YoloFace...")
            yf_min_e = minE.min_model_eps(data.clone().detach(), data_grad.clone().detach(), minE.yf_det_fn, (x1, y1, x2, y2))
            print("yunet min:", yn_min_e, "mediapipe min:", mp_min_e, "yoloface min:", yf_min_e)
            # Call FGSM Attack
            perturbed_data = fgsm_attack(data.clone().detach(), 0.2, data_grad.clone().detach(), x1, y1, x2, y2)
#             perturbed_data = fgsm_attack(data, max(yn_min_e, mp_min_e), data_grad) #data is the input image, epsilon
#             print("can detect faces on unperturbed img?", minE.mp_det_fn(data.detach()))
#             print(f"can detect faces on perturbed data with e={max(yn_min_e, mp_min_e) - 0.01}?", minE.mp_det_fn(fgsm_attack(data, max(yn_min_e, mp_min_e) - 0.01, data_grad).detach()))
#             print(f"can detect faces on perturbed img? with e={max(yn_min_e, mp_min_e) - 0.01}", minE.mp_det_fn(perturbed_data.detach()))
            
            df = df.append(row, ignore_index=True) #append the attributes of one face to the dataframe
            
            print('Perturbed')
            plt.imshow(tensor_to_image(perturbed_data))
            plt.show()
            
    df.to_csv(os.path.join(FOLDER_PATH, 'dataset' + str(int(time.time())) + '.csv'), index=False)  #save to csv

In [ ]:
pipeline(model, device)

In [ ]:
# it should not be able to detect this (but it can x_x)
def mp_det_fn(image):
    with minE.mp_face_detection.FaceDetection(min_detection_confidence=0.9, model_selection=0) as face_detection:
        results = face_detection.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        return results.detections is not None
mp_det_fn(cv2.imread("_2cantdetect.jpg"))

In [ ]:
import csv
import cv2
import itertools
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import skimage.feature as feature
data = cv2.imread("_2cantdetect.jpg")  # read the image
data = cv2.cvtColor(data, cv2.COLOR_BGR2RGB) #change to rgb
extract_image_attributes("aaa", 11111, data)